In [2]:
import collections
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import sklearn
import seaborn as sns
from sklearn import svm
from my_rbm import Rbm
import six.moves.cPickle as pickle
import sys
from pandas import *
from sklearn.preprocessing import OneHotEncoder
%matplotlib inline

In [10]:
def unpickle(file):
    fo = open(file, 'rb')
    dict = pickle.load(fo, encoding='latin-1')
    fo.close()
    return dict

def from_flat_to_3d(image):
#     print(image.shape)
    return np.dstack((image[0:1024].reshape(32,32),
                       image[1024:2048].reshape(32,32),
                       image[2048:3072].reshape(32,32)))

cifar_test = unpickle('cifar-10-batches-py/test_batch')
cifar_test['data'] = cifar_test['data'].astype(np.float32) / 255
cifar_test['data_3d'] = np.array([from_flat_to_3d(image) for image in cifar_test['data']])

cifar = unpickle('cifar-10-batches-py/data_batch_1')
for i in range(2, 6):
    tmp = unpickle('cifar-10-batches-py/data_batch_' + str(i))
    cifar['data'] = np.vstack((cifar['data'], tmp['data']))
    cifar['labels'] = np.concatenate((cifar['labels'], tmp['labels']))

cifar['data'] = cifar['data'].astype(np.float32) / 255
cifar['data_3d'] = np.array([from_flat_to_3d(image) for image in cifar['data']])

# cifar['data_bw'] = (cifar['data'][:,0:1024] + cifar['data'][:,1024:2048] + cifar['data'][:, 2048:3072]) / 3 
# cifar_test['data_bw'] = (cifar_test['data'][:,0:1024] + cifar_test['data'][:,1024:2048] + cifar_test['data'][:, 2048:3072]) / 3 

enc = OneHotEncoder()
cifar['labels_oh'] = enc.fit_transform(cifar['labels'].reshape(-1, 1))
cifar['labels_oh'] = cifar['labels_oh'].toarray()

cifar_test['labels'] = np.array(cifar_test['labels'])
cifar_test['labels_oh'] = enc.fit_transform(cifar_test['labels'].reshape(-1, 1))
cifar_test['labels_oh'] = cifar_test['labels_oh'].toarray()

# pca = PCA(whiten=True)
# cifar['data_bw_whitened'] = pca.fit_transform(cifar['data_bw'])
# cifar_test['data_bw_whitened'] = pca.fit_transform(cifar_test['data_bw'])

In [11]:
cifar_r = cifar['data'][:,0:1024]
cifar_b = cifar['data'][:,1024:2048]
cifar_g = cifar['data'][:,2048:]
cifar_test_r = cifar_test['data'][:,0:1024]
cifar_test_b = cifar_test['data'][:,1024:2048]
cifar_test_g = cifar_test['data'][:,2048:]

In [12]:
num_hidden = 261
num_epochs=10
rbm_r = Rbm(num_hidden=num_hidden, num_classes=10, num_features=1024, learning_rate=0.01)
rbm_r.init_rbm()
rbm_r.fit(cifar_r, cifar_test_r, num_epochs=num_epochs)

num_hidden = 261
num_epochs=10
rbm_b = Rbm(num_hidden=num_hidden, num_classes=10, num_features=1024, learning_rate=0.01)
rbm_b.init_rbm()
rbm_b.fit(cifar_b, cifar_test_b, num_epochs=num_epochs)

num_hidden = 261
num_epochs=10
rbm_g = Rbm(num_hidden=num_hidden, num_classes=10, num_features=1024, learning_rate=0.01)
rbm_g.init_rbm()
rbm_g.fit(cifar_g, cifar_test_g, num_epochs=num_epochs)

Number of features: 1024
Number of classes: 10
logit shape:  (?, 10)
batch_labels shape:  (?, 10)
epoch: 0
batch_number: 0
batch_number: 1000
batch_number: 2000
batch_number: 3000
batch_number: 4000
rec_loss: 0.155813
epoch: 1
batch_number: 0
batch_number: 1000
batch_number: 2000
batch_number: 3000
batch_number: 4000
rec_loss: 0.140213
epoch: 2
batch_number: 0
batch_number: 1000
batch_number: 2000
batch_number: 3000
batch_number: 4000
rec_loss: 0.12607
epoch: 3
batch_number: 0
batch_number: 1000
batch_number: 2000
batch_number: 3000
batch_number: 4000
rec_loss: 0.113225
epoch: 4
batch_number: 0
batch_number: 1000
batch_number: 2000
batch_number: 3000
batch_number: 4000
rec_loss: 0.105468
epoch: 5
batch_number: 0
batch_number: 1000
batch_number: 2000
batch_number: 3000
batch_number: 4000
rec_loss: 0.102192
epoch: 6
batch_number: 0
batch_number: 1000
batch_number: 2000
batch_number: 3000
batch_number: 4000
rec_loss: 0.0995677
epoch: 7
batch_number: 0
batch_number: 1000
batch_number: 2000

In [13]:
# Red
last_output_r = np.empty((0, rbm_r.get_h_prob_out([cifar['data'][0, 0:1024]]).shape[1]))
for i in range(cifar['data'].shape[0]):
    last_output_r = np.vstack((last_output_r, rbm_r.get_h_prob_out([cifar['data'][i, 0:1024]])))
    if i % 1000 == 0:
        print('done with: %s' % i)
        
last_output_test_r = np.empty((0, rbm_r.get_h_prob_out([cifar_test['data'][0, 0:1024]]).shape[1]))
for i in range(cifar_test['data'].shape[0]):
    last_output_test_r = np.vstack((last_output_test_r, rbm_r.get_h_prob_out([cifar_test['data'][i, 0:1024]])))
    if i % 999 == 0:
        print('done with: %s' % (i + 1))

# Blue
last_output_b = np.empty((0, rbm_b.get_h_prob_out([cifar['data'][0, 1024:2048]]).shape[1]))
for i in range(cifar['data'].shape[0]):
    last_output_b = np.vstack((last_output_b, rbm_b.get_h_prob_out([cifar['data'][i, 1024:2048]])))
    if i % 1000 == 0:
        print('done with: %s' % i)
        
last_output_test_b = np.empty((0, rbm_b.get_h_prob_out([cifar_test['data'][0, 1024:2048]]).shape[1]))
for i in range(cifar_test['data'].shape[0]):
    last_output_test_b = np.vstack((last_output_test_b, rbm_b.get_h_prob_out([cifar_test['data'][i, 1024:2048]])))
    if i % 999 == 0:
        print('done with: %s' % (i + 1))
        

        
# Green
last_output_g = np.empty((0, rbm_g.get_h_prob_out([cifar['data'][0, 2048:]]).shape[1]))
for i in range(cifar['data'].shape[0]):
    last_output_g = np.vstack((last_output_g, rbm_g.get_h_prob_out([cifar['data'][i, 2048:]])))
    if i % 1000 == 0:
        print('done with: %s' % i)
        
last_output_test_g = np.empty((0, rbm_g.get_h_prob_out([cifar_test['data'][0, 2048:]]).shape[1]))
for i in range(cifar_test['data'].shape[0]):
    last_output_test_g = np.vstack((last_output_test_g, rbm_g.get_h_prob_out([cifar_test['data'][i, 2048:]])))
    if i % 999 == 0:
        print('done with: %s' % (i + 1))

done with: 0
done with: 1000
done with: 2000
done with: 3000
done with: 4000
done with: 5000
done with: 6000
done with: 7000
done with: 8000
done with: 9000
done with: 10000
done with: 11000
done with: 12000
done with: 13000
done with: 14000
done with: 15000
done with: 16000
done with: 17000
done with: 18000
done with: 19000
done with: 20000
done with: 21000
done with: 22000
done with: 23000
done with: 24000
done with: 25000
done with: 26000
done with: 27000
done with: 28000
done with: 29000
done with: 30000
done with: 31000
done with: 32000
done with: 33000
done with: 34000
done with: 35000
done with: 36000
done with: 37000
done with: 38000
done with: 39000
done with: 40000
done with: 41000
done with: 42000
done with: 43000
done with: 44000
done with: 45000
done with: 46000
done with: 47000
done with: 48000
done with: 49000
done with: 1
done with: 1000
done with: 1999
done with: 2998
done with: 3997
done with: 4996
done with: 5995
done with: 6994
done with: 7993
done with: 8992
done w

In [14]:
stacked = np.hstack((last_output_r, last_output_g, last_output_b))
stacked_test = np.hstack((last_output_test_r, last_output_test_g, last_output_test_b))

In [ ]:
stacked.shape
stacked_test.shape

(10000, 783)

In [ ]:
clf = svm.SVC(decision_function_shape='ovr', kernel='poly', degree=3, coef0=1.0)
clf.fit(stacked, cifar['labels'])
clf.score(stacked_test, cifar_test['labels'])